# Custom Model Function Calling Evaluation Demo

**🎯 Goal**:
- Run a function calling evaluation in Okareo.
- Provide a simple introduction to Okareo evaluations.

**📋 Steps**:
1. Upload a function calling scenario.
2. Define a custom model to generate the function calls
3. Run the evaluation using the scenario (from #1) + model (from #2) along with checks to measure function call accuracy.

In [ ]:
# install okareo
%pip install okareo

In [ ]:
# get Okareo client
from okareo import Okareo

OKAREO_API_KEY = "<YOUR_OKAREO_API_KEY>"
okareo = Okareo(OKAREO_API_KEY)

Upload a simple scenario. Each row of the `seed_data` should contain:

- `input_`: query to the agent.
- `result`: the expected function call that answers the query.

In [ ]:
import random
import string

from okareo_api_client.models.scenario_set_create import ScenarioSetCreate
from okareo_api_client.models.seed_data import SeedData

def random_string(length: int) -> str:
    return "".join(random.choices(string.ascii_letters, k=length))

seed_data = [
    SeedData(
        input_="can you delete my account? my name is Bob",
        result={"name": "delete_account", "parameter_definitions": {"username": {"value": "Bob", "type": "str", "required": True}}},
    ),
    SeedData(
        input_="how do I make an account? I'm Alice",
        result={"name": "create_account", "parameter_definitions": {"username": {"value": "Alice", "type": "str", "required": True}}},
    ),
    SeedData(
        input_="how do I create an account?",
        result={"name": "create_account", "parameter_definitions": {"username": {"value": "Alice", "type": "str", "required": True}}},
    ),
    SeedData(
        input_="my name is John. how do I create a project?",
        result={"name": "create_account", "parameter_definitions": {"username": {"value": "Alice", "type": "str", "required": True}}},
    ),
]

tool_scenario = okareo.create_scenario_set(
    ScenarioSetCreate(
        name=f"Function Call Demo Scenario - {random_string(5)}",
        seed_data=seed_data,
    ) 
)

print(tool_scenario.app_link)

Define the [CustomModel](https://docs.okareo.ai/docs/sdk/okareo_python#custommodel--modelinvocation) using conditionals, and register the model with Okareo.

In reality, you would use the `CustomModel` class to invoke and parse your LLM's outputs. Feel free to play around!

In [ ]:
from okareo.model_under_test import CustomModel, ModelInvocation

class FunctionCallModel(CustomModel):
    def __init__(self, name):
        super().__init__(name)
        self.usernames = ["Bob", "Alice", "John"]

    def invoke(self, input_value):
        out = {"tool_calls": []}
        tool_call = {"name": "unknown"}

        # parse out the function name
        if "delete" in input_value:
            tool_call["name"] = "delete_account"
        if "create" in input_value:
            tool_call["name"] = "create_account"

        # parse out the function parameter
        tool_call["parameters"] = {}
        for username in self.usernames:
            if username in input_value:
                tool_call["parameters"]["username"] = username
                break

        # package the tool call and return
        out["tool_calls"].append(tool_call)
        return ModelInvocation(
            model_prediction=out,
            model_input=input_value,
        )

# Register the model to use in the test run
mut_name="Function Call Demo Model"
model_under_test = okareo.register_model(
    name=mut_name,
    model=[FunctionCallModel(name=FunctionCallModel.__name__)],
    update=True
)

Run a [Generation evaluation](https://docs.okareo.ai/docs/guides/generation_overview) on the custom model. 

We use predefined [checks in Okareo](https://docs.okareo.ai/docs/getting-started/concepts/checks), and the selected checks reflect the [Evaluation Metrics](https://gorilla.cs.berkeley.edu/blogs/8_berkeley_function_calling_leaderboard.html#metrics) used in the Berkeley Tool Calling Leaderboard. These checks include:

- "Is Function Correct"
- "Are Required Parameters Present"
- "Are All Parameters Expected"
- "Do Parameter Values Match"

In [ ]:
# evaluation that uses the scenario, check, and model
from okareo_api_client.models.test_run_type import TestRunType

eval_name = f"Function Call Demo Evaluation"
evaluation = model_under_test.run_test(
    name=eval_name,
    scenario=tool_scenario.scenario_id,
    test_run_type=TestRunType.NL_GENERATION,
    checks=[
        "is_function_correct",
        "are_required_params_present",
        "are_all_params_expected",
        "do_param_values_match",
    ],
)
print(f"See results in Okareo: {evaluation.app_link}")